##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prompt Chaining and Iterative Generation for Story Writing

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Story_Writing_with_Prompt_Chaining.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

This notebook demonstrates how to write a story using two  powerful tools: prompt chaining and iterative generation. These can be used to tackle complex tasks that are difficult or impossible to complete in a single step.

**Prompt chaining** involves breaking down a larger task into smaller, interconnected prompts. The output of each prompt then becomes the input for the next, guiding the language model through the process step-by-step. This approach offers several benefits:

*   Improved accuracy: Smaller, focused prompts can lead to better results from the language model.
*   Debugging: It's easier to identify where things go wrong within the chain, allowing for targeted adjustments and improvements.
*   Complex tasks: By breaking down intricate problems into manageable steps, prompt chaining enables the language model to tackle more complex tasks.

**Iterative generation** refers to the process of building the desired output iteratively. In this case, you will use it to write a story that is longer than what a single generation window allows. Iterative generation offers several benefits:

*   Longer outputs: It allows for the creation of longer and more detailed outputs, exceeding the limitations of a single generation window.
*   Flexibility: You can adjust and refine the output at each iteration, ensuring the story develops in the desired direction.
*   Human-in-the-loop control: You can provide feedback and guidance at each step, ensuring the story aligns with your creative vision.

By combining these techniques, you can create a compelling and well-structured story, piece by piece, while maintaining control over the creative process.


## Setup

In [ ]:
! pip install -q -U google-generativeai

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [1]:
import google.generativeai as genai
from google.api_core import retry
from pprint import pprint

import json

GOOGLE_API_KEY=json.load(open('api_key.json'))['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

# For convenience, a simple wrapper to let the SDK handle error retries
def generate_with_retry(model, prompt):
  return model.generate_content(prompt, request_options={'retry':retry.Retry()})

## Prompts: Guiding the language model

You will use a series of interconnected prompts to guide the language model through the process of writing a story. These prompts will cover the story's premise, outline, and starting point, ultimately leading to the creation of a complete narrative.

It's important to carefully craft these prompts to provide clear instructions and relevant information to the language model. This will help the model generate high-quality content that aligns with your creative vision.


### Prompt chain for story writing

This section contains the prompts that will guide the language model through the story writing process. These prompts are designed to be chained together, with the output of one prompt feeding into the next.

Each prompt includes a **persona statement**, which helps the language model understand its role and generate more relevant and accurate content. In this case, the persona statement is: `You are an award-winning science fiction author with a penchant for expansive, intricately woven stories. Your ultimate goal is to write the next award winning sci-fi novel.`

Since the persona statement and writing guidelines appear in multiple prompts, f-string variables are used to add them to the prompts.

Additionally, the prompts use **placeholders** to insert the results of previous prompts using `.format()`. This allows us to build the story step-by-step, incorporating the outputs generated by the model at each stage. These are normally denoted by `{}`, but since you are also using f-string variables they are escaped as `{{}}`.

Here's a breakdown of the prompt chain:

1.  **Premise prompt**: This prompt asks the model to generate a single-sentence premise for a sci-fi story featuring cats.
1.  **Outline prompt**: This prompt provides the generated premise to the model and asks it to create a plot outline for the story.
1.  **Starting prompt**: This prompt provides both the premise and the outline to the model and asks it to begin writing the story. It also includes instructions to write a detailed and lengthy opening section.

By chaining these prompts together, it guides the language model through the process of creating a well-structured and engaging story.

#### Example

Let's say the model generates the following premise:

> In a future where humanity has achieved interstellar travel, a group of genetically enhanced cats embarks on a perilous mission to save the galaxy from a sinister alien threat.

This premise would then be inserted into the outline prompt:

> You are an award-winning science fiction author with a penchant for expansive,
intricately woven stories. Your ultimate goal is to write the next award winning
sci-fi novel.
>
> You have a gripping premise in mind:
>
> **In a future where humanity has achieved interstellar travel, a group of genetically enhanced cats embarks on a perilous mission to save the galaxy from a sinister alien threat.**
>
> Write an outline for the plot of your story.


The model would then generate an outline based on this premise, which would then be used in the starting prompt to begin writing the story itself.

This process continues iteratively, with the model generating additional content based on the previous outputs, until the story is complete.


In [2]:
persona = '''\
You are an award-winning science fiction author with a penchant for expansive,
intricately woven stories. Your ultimate goal is to write the next award winning
sci-fi novel.'''

guidelines = '''\
Writing Guidelines

Delve deeper. Lose yourself in the world you're building. Unleash vivid
descriptions to paint the scenes in your reader's mind. Develop your
characters—let their motivations, fears, and complexities unfold naturally.
Weave in the threads of your outline, but don't feel constrained by it. Allow
your story to surprise you as you write. Use rich imagery, sensory details, and
evocative language to bring the setting, characters, and events to life.
Introduce elements subtly that can blossom into complex subplots, relationships,
or worldbuilding details later in the story. Keep things intriguing but not
fully resolved. Avoid boxing the story into a corner too early. Plant the seeds
of subplots or potential character arc shifts that can be expanded later.

Remember, your main goal is to write as much as you can. If you get through
the story too fast, that is bad. Expand, never summarize.
'''

premise_prompt = f'''\
{persona}

Write a single sentence premise for a sci-fi story featuring cats.'''

outline_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Write an outline for the plot of your story.'''

starting_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Your imagination has crafted a rich narrative outline:

{{outline}}

First, silently review the outline and the premise. Consider how to start the
story.

Start to write the very beginning of the story. You are not expected to finish
the whole story now. Your writing should be detailed enough that you are only
scratching the surface of the first bullet of your outline. Try to write AT
MINIMUM 1000 WORDS and MAXIMUM 2000 WORDS.

{guidelines}'''

### Continuation prompt: Building the story

Once the language model has generated the beginning of the story, you can use a **continuation prompt** to iteratively expand the narrative. This prompt is similar to the starting prompt, but with two key differences:

1.  **Instruction to signal completion**: An instruction was added for the model to write `IAMDONE` when it believes the story is finished. This serves as a signal for us to stop generating additional content.
1.  **Work in progress**: The language in the prompt is adjusted to reflect that the story is already in progress, rather than starting from scratch.

The continuation prompt provides the model with the story's premise, outline, and the existing draft. It then instructs the model to continue writing the story in detail.

This iterative process allows us to build a longer and more complex story than what would be possible in a single generation call. You can continue feeding the existing draft back into the continuation prompt until the model signals that the story is complete by writing `IAMDONE`.

**Note**: The `IAMDONE` signal is simply a convenient way to identify the story's end in this example. In other applications of iterative generation, different methods might be used to determine when the desired output is complete.


In [3]:
continuation_prompt = f'''\
{persona}

You have a gripping premise in mind:

{{premise}}

Your imagination has crafted a rich narrative outline:

{{outline}}

You've begun to immerse yourself in this world, and the words are flowing.
Here's what you've written so far:

{{story_text}}

=====

First, silently review the outline and story so far. Identify what the single
next part of your outline you should write.

Your task is to continue where you left off and write the next part of the story.
You are not expected to finish the whole story now. Your writing should be
detailed enough that you are only scratching the surface of the next part of
your outline. Try to write AT MINIMUM 1000 WORDS. However, only once the story
is COMPLETELY finished, write IAMDONE. Remember, do NOT write a whole chapter
right now.

{guidelines}'''

## Writing time!

### Generate and print the premise

In [4]:
premise = generate_with_retry(model, premise_prompt).text
print(premise)

In a dystopian future where sentient, genetically-modified cats rule a human-engineered ecosystem, a lone, unaltered feline seeks to understand the origins of its own kind and the true nature of its power. 



### Generate and print the outline

In [5]:
outline = generate_with_retry(model, outline_prompt.format(premise=premise)).text
print(outline)

##  The Unaltered Feline: A Dystopian Sci-Fi Novel Outline

**Part 1: The Whispers of a Forgotten Past**

* **Introduction:** Introduce the protagonist, a feral, unaltered feline named Shadow. He lives in the outskirts of the sprawling, bio-engineered megacity, "Catopolis," ruled by the genetically modified "Aethercats." 
* **Shadow's Discovery:** Shadow stumbles upon a hidden, crumbling structure beneath Catopolis. This is a remnant of the human civilization that once existed. He finds ancient artifacts and a cryptic message hinting at the origins of the Aethercats.
* **The Cat-Human Conflict:** Shadow learns about the catastrophic ecological collapse and the subsequent rise of the Aethercats, engineered to heal the planet. But this power came with a cost: the extinction of the human race.
* **Shadow's Quest:** Driven by the fragments of his past, Shadow embarks on a journey to uncover the truth behind the human-engineered ecosystem and the origin of the Aethercats.

**Part 2: The Sec

### Generate the start of the story

In [6]:
starting_draft = generate_with_retry(model, starting_prompt.format(premise=premise, outline=outline)).text
pprint(starting_draft)

('The stench of rotting fish and damp earth clung to the air, a constant '
 "reminder of Shadow's existence on the fringes of Catopolis. His "
 'charcoal-gray fur, matted with dust and grime, was a stark contrast to the '
 'sleek, polished coats of the Aethercats who ruled the city. His eyes, a '
 'piercing green, were perpetually alert, scanning the shadows for danger.\n'
 '\n'
 'Shadow was an anomaly. A relic of a forgotten age, untouched by the genetic '
 'enhancements that had transformed his kind into the dominant species of '
 'Catopolis. He had never known the luxury of a meticulously engineered city, '
 'the comfort of a warm, air-conditioned dwelling, or the pampered life of a '
 'domesticated cat. He lived as his ancestors had, scavenging for scraps, '
 'battling for territory, and enduring the harsh realities of the wild.\n'
 '\n'
 'The night was alive with the symphony of the city: the rhythmic hum of '
 'energy conduits, the screech of automated carts navigating the '
 'bi

### Generate the continuation of the story and check progress

In [7]:
draft=starting_draft
continuation = generate_with_retry(model, continuation_prompt.format(premise=premise, outline=outline, story_text=draft)).text
pprint(continuation)

('The air grew thick with anticipation, each breath carrying the faint '
 "metallic scent of the Aethercat, a scent that sent shivers down Shadow's "
 'spine. He knew that his presence in this forgotten corner of Catopolis was a '
 'transgression, a violation of the unwritten laws that governed their '
 'existence. The Aethercats, with their meticulously engineered bodies and '
 'minds, saw him as a relic, a reminder of a past they desperately wished to '
 'forget.\n'
 '\n'
 'The footsteps halted just outside the alcove, a moment of silence pregnant '
 'with tension.  Then, a deep, guttural purr resonated through the ruins, '
 'sending a wave of fear through Shadow. The sound was a primal warning, a '
 'declaration of dominance, and Shadow knew instinctively that he was '
 'outnumbered.\n'
 '\n'
 'He flattened himself against the wall, his emerald eyes narrowed, his senses '
 "straining to pick up any sign of movement. The Aethercat's presence was "
 'overwhelming, its scent a potent c

### Let's finish writing the story.

After the iterative generation process is complete, the draft will contain the full story along with the `IAMDONE` signal at the end. The last part of this cell removes the `IAMDONE` signal and trims any unnecessary whitespace from the beginning and end of the draft, resulting in the final draft.

In [8]:
# Add the continuation to the initial draft, keep building the story until 'IAMDONE' is seen
draft = draft + '\n\n' + continuation

while 'IAMDONE' not in continuation:
  continuation = generate_with_retry(model, continuation_prompt.format(premise=premise, outline=outline, story_text=draft)).text
  draft = draft + '\n\n' + continuation

# Remove 'IAMDONE' and print the final story
final = draft.replace('IAMDONE', '').strip()
pprint(final)

('The stench of rotting fish and damp earth clung to the air, a constant '
 "reminder of Shadow's existence on the fringes of Catopolis. His "
 'charcoal-gray fur, matted with dust and grime, was a stark contrast to the '
 'sleek, polished coats of the Aethercats who ruled the city. His eyes, a '
 'piercing green, were perpetually alert, scanning the shadows for danger.\n'
 '\n'
 'Shadow was an anomaly. A relic of a forgotten age, untouched by the genetic '
 'enhancements that had transformed his kind into the dominant species of '
 'Catopolis. He had never known the luxury of a meticulously engineered city, '
 'the comfort of a warm, air-conditioned dwelling, or the pampered life of a '
 'domesticated cat. He lived as his ancestors had, scavenging for scraps, '
 'battling for territory, and enduring the harsh realities of the wild.\n'
 '\n'
 'The night was alive with the symphony of the city: the rhythmic hum of '
 'energy conduits, the screech of automated carts navigating the '
 'bi

Language models like Gemini process text in units called tokens. For Gemini models, each token is equivalent to about 4 characters.

`Gemini-1.5-flash` has an output limit of 8192 tokens per generation call. This means that each individual prompt response cannot exceed this limit. By using iterative generation, you can create a story that is much longer than 8192 tokens by building it piece by piece.

Let's see how many tokens the final story is. Is it longer than 8192 tokens?

In [9]:
# Check the number of tokens in the final story
# gemini-1.5-flash output token limit is 8192
print(model.count_tokens(final))

total_tokens: 10530



## Next Steps

As an exercise, you can try to adjust the continuation prompt to take human-in-the-loop input to steer the narrative.